```stata

use "https://github.com/muzaale/ikesa/raw/main/nhanes.dta", clear
di "obs: `c(N)', vars: `c(k)'"
global subgroup ridreth3
global subgroupvar: var lab ridreth3
cls
gen years = permth_exm / 12
stset years, fail(mortstat)

#delimit ;
sts graph if inlist($subgroup,1,2,3,4,6,7),
    by($subgroup)
    fail
    ti("Mortality in NHANES III",pos(11))
    subti("by self report: ${subgroupvar}",pos(11))
    yti("%",orientation(horizontal))
    xti("Years")
    per(100)
    ylab(0(5)20,
        format(%3.0f)
        angle(360)
    )
    legend(on
        lab(1 "Mexican")
        lab(2 "Hispanic")
        lab(3 "White")
        lab(4 "Black")
        lab(5 "Asian")
        lab(6 "Other")
        ring(0)
        pos(11)
        col(1)
        order(3 4 1 2  5)
    )
    note("Source: RDC/NCHS/CDC/DHHS")  
;
#delimit cr
cd "~/dropbox/1f.ἡἔρις,κ/1.ontology/alpha"
global confounders ridageyr diq010 bmxbmi smq020 lbdscrsi lbxgh
stcox i.$subgroup $confounders if inlist(${subgroup}, 1, 2, 3, 4, 6, 7), basesurv(s0) //best when centered
matrix define m = r(table)
matrix b = e(b)
matrix V = e(V)
desc $subgroup $confounders
matrix list m
//SV: black, 40yo, diabetic, BMI=36, h/o smoking, SCr=1.8, HbA1c=7.1
matrix SV = (0, 0, 0, 1, 0, 0, 60, 1, 36, 1, 1.5, 7.1)
matrix risk_score = SV * b'
//log HR for scenario vector above
matrix list risk_score 
//HR for scenario described compared with "base-case" (would make sense if there was centering. but without centering the base-case is for Mexican, 0yo, non-diabetic, BMI=0, etc.)
di exp(risk_score[1,1])
matrix var_prediction = SV * V * SV'
matrix se_prediction = sqrt(var_prediction[1,1])
//10-year mortality for scenario 
gen f0 = (1 - s0) * 100
gen f1 = f0 * exp(risk_score[1,1])
drop if _t > 10
line f1 _t, sort connect(step step) ylab(0(5)20) xlab(0(2)10)
graph export nhanes_scenario.png, replace  
```

Let's delve into the specific portion of the code that deals with the `risk score calculation` and the scenario vector (SV). I'll explain each part of this code snippet and highlight the specific example provided:

1. **Defining Matrices and Risk Score:**
   ```stata
   matrix define m = r(table)
   matrix b = e(b)
   matrix V = e(V)
   ```
   These lines define matrices based on the results of the previously fit Cox model. `m` stores the table of results, `b` stores the coefficients, and `V` stores the variance-covariance matrix.

2. **Scenario Vector (SV):**
   ```stata
   //SV: black, 40yo, diabetic, BMI=36, h/o smoking, SCr=1.8, HbA1c=7.1
   matrix SV = (0, 0, 0, 1, 0, 0, 60, 1, 36, 1, 1.5, 7.1)
   ```
   Here, the SV is defined with specific values for an individual scenario. In this example, the scenario represents a 40-year-old black individual who is diabetic, has a BMI of 36, a history of smoking, a serum creatinine (SCr) level of 1.8, and a Hemoglobin A1c (HbA1c) of 7.1.

3. **Calculating the Risk Score:**
   ```stata
   matrix risk_score = SV * b'
   ```
   This line multiplies the scenario vector with the transpose of the coefficients to calculate the risk score.

4. **Log Hazard Ratio:**
   ```stata
   //log HR for scenario vector above
   matrix list risk_score 
   ```
   This displays the log hazard ratio for the specified scenario.

5. **Hazard Ratio for Scenario:**
   ```stata
   //HR for scenario described compared with "base-case"
   di exp(risk_score[1,1])
   ```
   This calculates the hazard ratio by taking the exponential of the log hazard ratio.

6. **Variance and Standard Error of Prediction:**
   ```stata
   matrix var_prediction = SV * V * SV'
   matrix se_prediction = sqrt(var_prediction[1,1])
   ```
   These lines calculate the variance and standard error of the prediction for the scenario.

7. **Comment About Base-Case:**
   The comment explains that the interpretation would be more intuitive if centering were used. Without centering, the base-case is for a specific reference group (e.g., Mexican, 0 years old, non-diabetic, BMI=0, etc.)

To adapt this portion of the code for another project or scenario, the analyst would need to redefine the scenario vector (`SV`) with the appropriate values for the new context. They would also need to ensure that the model's coefficients and variance-covariance matrix are correctly extracted, and that the calculations are aligned with the new scenario's specifications.

![](https://github.com/jhutrc/beta/blob/main/nhanes_scenario.png?raw=true)